### Data format corrections

In [1]:
import numpy as np
import pandas as pd

import datetime
from collections import defaultdict
import pybgpstream

In [2]:
# import modin.pandas as pd
# import ray
# ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})

In [36]:
dfr = pd.read_hdf("../data/01_raw/googletest-small.h5", key='raw')

In [37]:
dfr

,type,time,peer_asn,peer_ip,prefix,as_path
0,A,1.396368e+09,25152,202.249.2.185,177.107.11.0/24,25152 2914 1239 16735 53059 262730
1,A,1.396368e+09,25152,202.249.2.185,190.211.244.0/24,25152 6939 12956 22927 52295
2,A,1.396368e+09,25152,202.249.2.185,190.211.245.0/24,25152 6939 12956 22927 52295
3,A,1.396368e+09,25152,202.249.2.185,177.107.11.0/24,25152 6939 16735 53059 262730
4,A,1.396368e+09,25152,202.249.2.185,177.107.11.0/24,25152 6939 3549 16735 53059 262730
...,...,...,...,...,...,...
20704,A,1.396372e+09,25152,202.249.2.185,94.139.170.0/23,25152 6939 8529 12880 25124 57218 57218 57218
20705,A,1.396372e+09,25152,202.249.2.185,188.253.120.0/22,25152 6939 8529 48159 58127
20706,A,1.396372e+09,25152,202.249.2.185,5.233.125.0/24,25152 6939 8529 48159
20707,A,1.396372e+09,25152,202.249.2.185,66.210.60.0/24,25152 6939 22773 20450


In [38]:
dfr = dfr.drop(columns=["peer_ip"])

In [39]:
# r.time
# datetime.datetime.utcfromtimestamp(r.time).strftime('%Y-%m-%d %H:%M:%S.%f')
dfr.time = pd.to_datetime(dfr["time"]*10e8, unit='ns')

In [40]:
dfr["type"] = dfr.apply(lambda x: True if x.type == "A" else False, axis=1)

In [41]:
dfr["peer_asn"] = dfr["peer_asn"].astype("category")

In [42]:
dfr["prefix_ip"], dfr["prefix_nm"] = zip(*dfr["prefix"].apply(lambda x: x.split('/')))

In [10]:
# dfr = dfr.astype({"peer_asn": np.uint32,"prefix_nm":np.uint8 })
dfr = dfr.astype({"prefix_nm":np.uint8 })

In [11]:
# np.array(dfr["peer_ip"].apply(lambda x: x.split('.')), dtype=np.uint8)
# print(set(zip(*dfr["peer_ip"].apply(lambda x: x.split('.')))))

In [12]:
# dftmp = pd.DataFrame(dfr["prefix_ip"].apply(lambda x: np.array(x.split('.'),dtype=np.uint8 )).tolist())
# dfr["peer_ip_a"], dfr["peer_ip_b"], dfr["peer_ip_c"], dfr["peer_ip_d"],  = zip(*dfr["peer_ip"].apply(lambda x: np.array(x.split('.'))))

#disabled temp
# dfr["prefix_ip_a"], dfr["prefix_ip_b"], dfr["prefix_ip_c"], dfr["prefix_ip_d"]  = zip(*dfr["prefix_ip"].apply(lambda x: np.array(x.split('.'))))
# dfr["prefix_ip_a"] = dfr["prefix_ip_a"].astype(np.uint8)
# dfr["prefix_ip_b"] = dfr["prefix_ip_b"].astype(np.uint8)
# dfr["prefix_ip_c"] = dfr["prefix_ip_c"].astype(np.uint8)
# dfr["prefix_ip_d"] = dfr["prefix_ip_d"].astype(np.uint8)

In [13]:
# def ipstr2ipint(ipstr):
#     parts = ipstr.split('.')
#     return (int(parts[0]) << 24) + (int(parts[1]) << 16) + (int(parts[2]) << 8) + int(parts[3])

In [14]:
# from cyberpandas import to_ipaddress
# # dfr["peer_ip"] = to_ipaddress(dfr["peer_ip"])
# dfr["prefix_ip"] = to_ipaddress(dfr["prefix_ip"])
# dfr["prefix_ip_int"] = dfr["prefix_ip"].apply(lambda x: int(x))

In [15]:
dfr["prefix_ip_cat"] = dfr["prefix"].astype("category")

In [16]:
# dfr = dfr.drop(columns=["prefix"])


In [17]:
dfr.loc[dfr["prefix_nm"] == 25]

,type,time,peer_asn,prefix,as_path,prefix_ip,prefix_nm,prefix_ip_cat
10529,True,2014-04-01 16:02:26,3333,91.212.118.0/25,3333 9002 5563 49208 49208,91.212.118.0,25,91.212.118.0/25
10531,True,2014-04-01 16:02:26,8758,91.212.118.0/25,8758 8758 9002 5563 49208 49208,91.212.118.0,25,91.212.118.0/25
13957,True,2014-04-01 16:03:23,57381,91.212.118.0/25,57381 42708 9002 5563 49208 49208,91.212.118.0,25,91.212.118.0/25
14864,True,2014-04-01 16:03:33,3333,46.22.126.128/25,3333 50683 50989,46.22.126.128,25,46.22.126.128/25
14878,True,2014-04-01 16:03:33,3333,46.22.115.0/25,3333 50683 50989,46.22.115.0,25,46.22.115.0/25
...,...,...,...,...,...,...,...,...
324592,True,2014-04-01 16:57:54,3333,46.22.115.0/25,3333 50683 50989,46.22.115.0,25,46.22.115.0/25
328881,True,2014-04-01 16:58:55,3333,46.22.115.0/25,3333 1103 42708 50683 50989,46.22.115.0,25,46.22.115.0/25
328882,True,2014-04-01 16:58:55,3333,46.22.126.128/25,3333 1103 42708 50683 50989,46.22.126.128,25,46.22.126.128/25
328960,True,2014-04-01 16:58:55,3333,46.22.126.128/25,3333 42708 50683 50989,46.22.126.128,25,46.22.126.128/25


In [18]:
# dfr[["type", 	"time" ,	"peer_asn" ,	"as_path" 	, 	"prefix_nm" ,	"prefix_ip_a" ,	"prefix_ip_b" ,	"prefix_ip_c", 	"prefix_ip_d", 	"prefix_ip_cat"]]
dfr

,type,time,peer_asn,prefix,as_path,prefix_ip,prefix_nm,prefix_ip_cat
0,True,2014-04-01 16:00:00,7018,91.229.226.0/24,7018 174 15772 15772 56978,91.229.226.0,24,91.229.226.0/24
1,True,2014-04-01 16:00:00,9304,195.190.132.0/24,9304 6453 3491 12389 35400 39240,195.190.132.0,24,195.190.132.0/24
2,True,2014-04-01 16:00:00,9304,46.167.72.0/21,9304 6453 3491 12389 35400 8427,46.167.72.0,21,46.167.72.0/21
3,True,2014-04-01 16:00:00,9304,46.167.116.0/22,9304 6453 3491 12389 35400 8427,46.167.116.0,22,46.167.116.0/22
4,True,2014-04-01 16:00:00,9304,79.134.30.0/23,9304 6453 3491 12389 35400 8427,79.134.30.0,23,79.134.30.0/23
...,...,...,...,...,...,...,...,...
332520,True,2014-04-01 17:00:00,7018,37.236.32.0/21,7018 3356 47887 50710,37.236.32.0,21,37.236.32.0/21
332521,True,2014-04-01 17:00:00,7018,37.237.248.0/21,7018 3356 47887 50710,37.237.248.0,21,37.237.248.0/21
332522,True,2014-04-01 17:00:00,7018,37.236.248.0/21,7018 3356 47887 50710,37.236.248.0,21,37.236.248.0/21
332523,True,2014-04-01 17:00:00,7018,37.237.24.0/21,7018 3356 47887 50710,37.237.24.0,21,37.237.24.0/21


## Path

In [19]:
# t = "25152 6939 23520 26617 26617 26617 {65460,65470}"
# import re
# #re.split(' |,|{|}',t)

# path_pattern = re.compile('(\d+)+')
# path_str = path_pattern.findall(t)
# path_np = np.array(path_str, dtype=np.uint32)

# path_np

In [20]:
import re
path_pattern = re.compile('(\d+)+')
# asp='1836 174 7922 33287 {21669}'
# 25152 6939 23520 26617 26617 26617 {65460,65470}
# asp.replace('{','').replace('}','').split(' ')
def process_as_path(asp):
    if type(asp) == np.ndarray:
        return asp, 0
    else:
        # print(asp)
        # npasp = np.array(asp.replace('{','').replace('}','').split(' '), dtype=np.uint32)
        # npasp = np.array(asp.replace('{','').replace('}','').split(' '), dtype=np.uint32)
        path_str = path_pattern.findall(asp)
        npasp = np.array(path_str, dtype=np.uint32)
        return npasp, len(npasp)

In [21]:
# dfr[dfr["type"] == True]["as_path"].apply(lambda x: x.replace('{','').replace('}','').split(' '))
# dfr["as_path"] = dfr[dfr["type"] == True]["as_path"].apply(lambda x: np.array(x.replace('{','').replace('}','').split(' '), dtype=np.uint32))
# dfr["as_path_len"] = dfr[dfr["type"] == True]["as_path"].apply(lambda x: len(x))

In [22]:
dfr["as_path"], dfr["as_path_len"] = zip(*dfr["as_path"].apply(process_as_path))

In [23]:
# dfr["as_path"], dfr["as_path_len"] = zip(*dfr[dfr["type"] == True]["as_path"].apply(process_as_path))

In [24]:
dfr

,type,time,peer_asn,prefix,as_path,prefix_ip,prefix_nm,prefix_ip_cat,as_path_len
0,True,2014-04-01 16:00:00,7018,91.229.226.0/24,"[7018, 174, 15772, 15772, 56978]",91.229.226.0,24,91.229.226.0/24,5
1,True,2014-04-01 16:00:00,9304,195.190.132.0/24,"[9304, 6453, 3491, 12389, 35400, 39240]",195.190.132.0,24,195.190.132.0/24,6
2,True,2014-04-01 16:00:00,9304,46.167.72.0/21,"[9304, 6453, 3491, 12389, 35400, 8427]",46.167.72.0,21,46.167.72.0/21,6
3,True,2014-04-01 16:00:00,9304,46.167.116.0/22,"[9304, 6453, 3491, 12389, 35400, 8427]",46.167.116.0,22,46.167.116.0/22,6
4,True,2014-04-01 16:00:00,9304,79.134.30.0/23,"[9304, 6453, 3491, 12389, 35400, 8427]",79.134.30.0,23,79.134.30.0/23,6
...,...,...,...,...,...,...,...,...,...
332520,True,2014-04-01 17:00:00,7018,37.236.32.0/21,"[7018, 3356, 47887, 50710]",37.236.32.0,21,37.236.32.0/21,4
332521,True,2014-04-01 17:00:00,7018,37.237.248.0/21,"[7018, 3356, 47887, 50710]",37.237.248.0,21,37.237.248.0/21,4
332522,True,2014-04-01 17:00:00,7018,37.236.248.0/21,"[7018, 3356, 47887, 50710]",37.236.248.0,21,37.236.248.0/21,4
332523,True,2014-04-01 17:00:00,7018,37.237.24.0/21,"[7018, 3356, 47887, 50710]",37.237.24.0,21,37.237.24.0/21,4


In [25]:
# dfr["as_path"].apply(lambda x: type(x)).value_counts()

In [26]:
# dfr.to_hdf("googletest-raw.h5", key='preprocessed', orient="table")
dfr.to_json("../data/02_intermediate/googletest.json", orient="table")
# dfr.to_hdf("googletest-raw.h5", key='path', format="table")

## Prefix one-hot encoding 

In [27]:
dfr.memory_usage()
dfr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 332525 entries, 0 to 332524
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   type           332525 non-null  bool          
 1   time           332525 non-null  datetime64[ns]
 2   peer_asn       332525 non-null  category      
 3   prefix         332525 non-null  object        
 4   as_path        332525 non-null  object        
 5   prefix_ip      332525 non-null  object        
 6   prefix_nm      332525 non-null  uint8         
 7   prefix_ip_cat  332525 non-null  category      
 8   as_path_len    332525 non-null  int64         
dtypes: bool(1), category(2), datetime64[ns](1), int64(1), object(3), uint8(1)
memory usage: 25.5+ MB


In [28]:
# from sklearn import preprocessing
# lb = preprocessing.LabelBinarizer()
# y = dfr["prefix_ip"]
# ylab = lb.fit(y)
# df = pd.DataFrame()

In [29]:
# from sklearn import preprocessing
# lb = preprocessing.LabelBinarizer()
# dfr = dfr.join(pd.DataFrame(lb.fit_transform(dfr["prefix_ip"]), columns=lb.classes_, index=dfr.index))

In [30]:
# oneliner:
# dfr = dfr.join(pd.DataFrame(lb.fit_transform(dfr["prefix_ip"]), columns=lb.classes_, index=dfr.index))

In [33]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer(sparse_output=True)
ylab = lb.fit(dfr["prefix_ip"])
predf = lb.transform(dfr["prefix_ip"])


In [34]:
predf

<332525x20696 sparse matrix of type '<class 'numpy.int64'>'
	with 332525 stored elements in Compressed Sparse Row format>

In [35]:
predfspar = pd.DataFrame(predf, columns=lb.classes_, index=dfr.index, dtype="Sparse[int8]")

/home/cmos/.virtualenvs/bgpcompute-kedro/lib/python3.10/site-packages/pandas/core/arrays/sparse/array.py:502: SparseEfficiencyWarning: Comparing a sparse matrix with 0 using == is inefficient, try using != instead.
  sparse_values, sparse_index, fill_value = make_sparse(


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all().

In [ ]:
predfspar.info()

In [ ]:
dfr = dfr.join(predfspar)

In [ ]:
dfr

In [ ]:
del(predfspar)

In [ ]:
# dfr.to_hdf("googletest-raw.h5", key="features", format="table")
dfr.to_csv("../data/04_feature/googletest-features.csv")

In [ ]:
from pandas_profiling import ProfileReport

profile = ProfileReport(dfr, minimal=True)
# profile = ProfileReport(dfr[["type", 	"time" ,	"peer_asn" ,	"as_path" 	, 	"prefix_nm" ,	"prefix_ip_a" ,	"prefix_ip_b" ,	"prefix_ip_c", 	"prefix_ip_d", 	"prefix_ip_cat"]], minimal=True)
profile.to_file("output.html")

